# CONFIGURATION

In [ ]:
%pip install peft accelerate


In [ ]:
# Cell [1] - ĐÃ CẬP NHẬT CHO PROMPT TUNING
import os
from peft import TaskType  # <<< THÊM IMPORT NÀY

MODEL_NAMES = [
    "joeddav/xlm-roberta-large-xnli",
    "microsoft/infoxlm-large",
    "uitnlp/CafeBERT",
    "FacebookAI/xlm-roberta-large",
    "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli",
    "MoritzLaurer/ernie-m-large-mnli-xnli",
    "microsoft/deberta-xlarge-mnli",
]


class Config:
    ROOT_DIR = os.getcwd()
    DATA_DIR = os.path.join(ROOT_DIR, "data")
    TRAIN_FILE = os.path.join(DATA_DIR, "vihallu-train.csv")
    TEST_FILE = os.path.join(DATA_DIR, "vihallu-public-test.csv")
    SUBMISSION_DIR = os.path.join(ROOT_DIR, "submission")
    SUBMISSION_CSV = "submit.csv"
    SUBMISSION_ZIP = "submit.zip"

    MODEL_NAME = MODEL_NAMES[3]
    # --- THAY ĐỔI TÊN THƯ MỤC OUTPUT CHO PEFT ---
    MODEL_OUTPUT_DIR = os.path.join(
        ROOT_DIR,
        "models",
        f"{MODEL_NAME.split('/')[-1]}-prompt-tuned",  # Sửa tên thư mục
    )

    MAX_LENGTH = 512
    RANDOM_STATE = 42
    EPOCHS = 10  # Có thể cần nhiều epoch hơn vì chỉ train prompt
    BATCH_SIZE = 4
    GRADIENT_ACCUMULATION_STEPS = 4

    # --- SỬ DỤNG CẤU HÌNH ỔN ĐỊNH CHO PROMPT ---
    SCHEDULER_TYPE = "cosine"  # Cosine đơn giản thường tốt cho PEFT
    LEARNING_RATE = 1e-3  # <<-- LR cao hơn nhiều cho prompt embeddings
    WEIGHT_DECAY = 0.01  # Giữ nguyên hoặc giảm nhẹ
    NUM_CYCLES = 3

    # --- CÁC THAM SỐ PEFT (PROMPT TUNING) ---
    PEFT_TYPE = "PROMPT_TUNING"
    PEFT_TASK_TYPE = TaskType.SEQ_CLS  # <<< CHO SEQUENCE CLASSIFICATION
    NUM_VIRTUAL_TOKENS = 20  # Số lượng token ảo để học (thường từ 10-100)

    # --- CÁC THAM SỐ ÍT QUAN TRỌNG HƠN KHI DÙNG PEFT ---
    # CLASSIFIER_DROPOUT = 0.1 # Ít ảnh hưởng vì classifier gốc bị đóng băng
    LABEL_SMOOTHING = 0.05
    EPSILON = 1e-8
    PATIENCE_LIMIT = 4  # Giữ patience an toàn
    TOTAL_STEP_SCALE = 0.1
    VALIDATION_SPLIT_SIZE = 0.2

    LABEL_MAP = {"no": 0, "extrinsic": 1, "intrinsic": 2}
    ID2LABEL = {v: k for k, v in LABEL_MAP.items()}
    CLASS_WEIGHTS = [1.0393466963622866, 1.0114145354717525, 0.9531590413943355]


cfg = Config()


# LOGGER

In [ ]:
import logging
import os
from datetime import datetime

# Thư mục gốc để lưu tất cả các file log
LOG_BASE_DIR = "logs"

# Dùng một dictionary để lưu các logger đã tạo, tránh việc tạo lại và gây ra log trùng lặp
_loggers = {}


def setup_logger(model_name: str, log_level=logging.INFO):
    """
    Thiết lập và trả về một logger để ghi log vào cả console và file.

    - Mỗi model sẽ có một thư mục log riêng dựa trên `model_name`.
    - Mỗi lần chạy sẽ tạo một file log mới có tên là timestamp (ví dụ: 2023-10-27_15-30-00.log).
    - Đảm bảo không có log nào bị ghi đè.

    Args:
        model_name (str): Tên của model, dùng để tạo thư mục con. Ví dụ: 'xnli-large-tuned'.
        log_level (int): Cấp độ log, mặc định là logging.INFO.

    Returns:
        logging.Logger: Instance của logger đã được cấu hình.
    """
    # Nếu logger cho model này đã tồn tại, trả về nó ngay lập tức
    if model_name in _loggers:
        return _loggers[model_name]

    # Xử lý tên model để an toàn khi tạo tên thư mục (thay thế "/")
    safe_model_name = model_name.replace("/", "_").replace("\\", "_")
    model_log_dir = os.path.join(LOG_BASE_DIR, safe_model_name)
    os.makedirs(model_log_dir, exist_ok=True)

    # Tạo logger
    logger = logging.getLogger(safe_model_name)
    logger.setLevel(log_level)

    # Ngăn không cho log lan truyền đến root logger để tránh in ra console 2 lần
    logger.propagate = False

    # Định dạng cho log message
    formatter = logging.Formatter(
        "%(asctime)s - [%(levelname)s] - %(message)s", datefmt="%Y-%m-%d %H:%M:%S"
    )

    # Tạo File Handler để ghi log ra file
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    log_file_path = os.path.join(model_log_dir, f"{timestamp}.log")

    file_handler = logging.FileHandler(log_file_path, encoding="utf-8")
    file_handler.setLevel(log_level)
    file_handler.setFormatter(formatter)

    # Tạo Console (Stream) Handler để in log ra màn hình
    console_handler = logging.StreamHandler()
    console_handler.setLevel(log_level)
    console_handler.setFormatter(formatter)

    # Thêm các handler vào logger
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    # Lưu logger vào cache
    _loggers[model_name] = logger

    logger.info(
        f"Logger cho '{safe_model_name}' đã được khởi tạo. File log: {log_file_path}"
    )

    return logger


## Setup logger

In [ ]:
logger = setup_logger(f"{cfg.MODEL_NAME}-training")
logger.info(f"Logger initialized for {cfg.MODEL_NAME}")

logger.info("=" * 60)
logger.info("🚀 STARTING TRAINING SESSION")
logger.info("=" * 60)
for key, value in Config.__dict__.items():
    if not key.startswith("__") and not callable(value):
        logger.info(f"{key}: {value}")
logger.info("=" * 60)


2025-10-17 13:42:24 - [INFO] - Logger cho 'FacebookAI_xlm-roberta-large-training' đã được khởi tạo. File log: logs/FacebookAI_xlm-roberta-large-training/2025-10-17_13-42-24.log
2025-10-17 13:42:24 - [INFO] - Logger initialized for FacebookAI/xlm-roberta-large
2025-10-17 13:42:24 - [INFO] - ============================================================
2025-10-17 13:42:24 - [INFO] - 🚀 STARTING TRAINING SESSION
2025-10-17 13:42:24 - [INFO] - ============================================================
2025-10-17 13:42:24 - [INFO] - ROOT_DIR: /home/guest/Projects/CS221
2025-10-17 13:42:24 - [INFO] - DATA_DIR: /home/guest/Projects/CS221/data
2025-10-17 13:42:24 - [INFO] - TRAIN_FILE: /home/guest/Projects/CS221/data/vihallu-train.csv
2025-10-17 13:42:24 - [INFO] - TEST_FILE: /home/guest/Projects/CS221/data/vihallu-public-test.csv
2025-10-17 13:42:24 - [INFO] - SUBMISSION_DIR: /home/guest/Projects/CS221/submission
2025-10-17 13:42:24 - [INFO] - SUBMISSION_CSV: submit.csv
2025-10-17 13:42:24 - 

# Hallucination Dataset

In [ ]:
# Cell [6] - ĐÃ CẬP NHẬT
import torch
from torch.utils.data import Dataset


class HallucinationDataset(Dataset):
    def __init__(self, premises, hypotheses, labels, tokenizer, max_len):
        self.premises = premises
        self.hypotheses = hypotheses
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        premise = self.premises[idx]
        hypothesis = self.hypotheses[idx]
        label = self.labels[idx]

        # Tokenize bằng cách truyền 2 chuỗi riêng biệt
        # Tokenizer sẽ tự động cắt bớt `premise` nếu cần
        encoding = self.tokenizer.encode_plus(
            premise,
            hypothesis,  # <-- text_pair
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            truncation=True,
            return_attention_mask=True,
        )

        return {
            "input_ids": encoding["input_ids"],
            "attention_mask": encoding["attention_mask"],
            "labels": label,
        }


## Prepare data

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd


def prepare_data(config, logger=None):
    df = pd.read_csv(config.TRAIN_FILE)
    print(f"✅ Đọc thành công {len(df)} mẫu từ file đã xử lý: {config.TRAIN_FILE}")

    # Tạo 2 cột premise và hypothesis từ ngữ cảnh (context)
    df["premise"] = (
        "Câu hỏi: "
        + df["prompt"].astype(str)
        + " Ngữ cảnh: "
        + df["context"].astype(str)
    )
    df["hypothesis"] = df["response"].astype(str)

    df["label_id"] = df["label"].map(config.LABEL_MAP)
    df.dropna(subset=["label_id"], inplace=True)
    df["label_id"] = df["label_id"].astype(int)

    train_df, val_df = train_test_split(
        df,
        test_size=config.VALIDATION_SPLIT_SIZE,
        random_state=config.RANDOM_STATE,
        stratify=df["label_id"],
    )

    if logger:
        logger.info(
            f"Chia dữ liệu: {len(train_df)} mẫu train, {len(val_df)} mẫu validation."
        )

    # --- PHẦN NÂNG CẤP: LƯU FILE RA THƯ MỤC DATA ---
    # Tạo thư mục 'processed' trong 'data' nếu chưa có
    processed_data_dir = os.path.join(config.DATA_DIR, "processed")
    os.makedirs(processed_data_dir, exist_ok=True)

    # Định nghĩa đường dẫn file
    train_output_path = os.path.join(processed_data_dir, "train_split.csv")
    val_output_path = os.path.join(processed_data_dir, "validation_split.csv")

    # Lưu các DataFrame
    train_df.to_csv(train_output_path, index=False, encoding="utf-8-sig")
    val_df.to_csv(val_output_path, index=False, encoding="utf-8-sig")

    print(f"✅ Đã lưu tập train vào: {train_output_path}")
    print(f"✅ Đã lưu tập validation vào: {val_output_path}")
    # --- KẾT THÚC PHẦN NÂNG CẤP ---

    return train_df, val_df


# Model

In [ ]:
# Cell [8] - ĐÃ CẬP NHẬT HOÀN TOÀN CHO PEFT

from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification
import torch.nn as nn
from peft import (
    get_peft_model,
    PromptTuningConfig,
    TaskType,
    PeftModel,
)  # <<< THÊM IMPORT PEFT


def get_model_and_tokenizer(config, logger):
    """Tải pre-trained model, tokenizer và áp dụng PEFT."""
    logger.info(f"Đang tải model gốc: {config.MODEL_NAME}")
    tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)

    # Tải model gốc cho sequence classification
    model = AutoModelForSequenceClassification.from_pretrained(
        config.MODEL_NAME,
        num_labels=len(config.LABEL_MAP),
        # Thêm các map label nếu cần (thường không cần cho PEFT vì head gốc bị đóng băng)
        # label2id=config.LABEL_MAP,
        # id2label=config.ID2LABEL,
    )

    # --- TẠO VÀ ÁP DỤNG PEFT CONFIG ---
    if config.PEFT_TYPE == "PROMPT_TUNING":
        logger.info(
            f"Áp dụng {config.PEFT_TYPE} với {config.NUM_VIRTUAL_TOKENS} virtual tokens."
        )
        peft_config = PromptTuningConfig(
            task_type=config.PEFT_TASK_TYPE,
            num_virtual_tokens=config.NUM_VIRTUAL_TOKENS,
            # Các tham số khác có thể thêm:
            # tokenizer_name_or_path=config.MODEL_NAME,
            # prompt_tuning_init="TEXT", # Khởi tạo prompt bằng text
            # prompt_tuning_init_text="Classify the hallucination type:", # Text dùng để khởi tạo
        )
    # Thêm các loại PEFT khác nếu muốn (LoRA, etc.)
    # elif config.PEFT_TYPE == "LORA":
    #    ...
    else:
        logger.warning(
            f"PEFT_TYPE '{config.PEFT_TYPE}' không được hỗ trợ hoặc không được chọn. Sử dụng model gốc."
        )
        return model, tokenizer  # Trả về model gốc nếu không dùng PEFT

    # Lấy mô hình đã được PEFT bao bọc
    model = get_peft_model(model, peft_config)

    logger.info("Đã áp dụng PEFT adapter thành công.")
    model.print_trainable_parameters()  # In ra số lượng tham số cần huấn luyện
    # ------------------------------------

    return model, tokenizer


# Training

In [ ]:
import os
import math
import torch
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from tqdm.auto import tqdm
from torch.optim import AdamW
from huggingface_hub import login
from transformers import get_scheduler
from torch.utils.data import DataLoader
from tqdm.contrib.logging import logging_redirect_tqdm
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, accuracy_score, classification_report
from functools import partial


## train one epoch function

In [ ]:
def train_one_epoch(
    model,
    data_loader,
    loss_fn,
    optimizer,
    scheduler,
    device,
    epoch=None,
    total_epochs=None,
    gradient_accumulation_steps=1,
):
    """Huấn luyện mô hình trong một epoch bằng gradient accumulation."""
    model.train()
    total_loss = 0
    desc = f"Train" if epoch is None else f"Epoch {epoch}/{total_epochs}"
    progress_bar = tqdm(
        data_loader, desc=desc, leave=False, dynamic_ncols=True, mininterval=0.5
    )

    optimizer.zero_grad()
    steps_in_epoch = len(data_loader)
    with logging_redirect_tqdm():  # make logger calls safe
        for step, batch in enumerate(progress_bar):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits  # shape (batch_size, num_labels)

            loss = loss_fn(logits, labels)
            total_loss += loss.item()
            scaled_loss = loss / gradient_accumulation_steps
            scaled_loss.backward()

            if (step + 1) % gradient_accumulation_steps == 0 or (
                step + 1
            ) == steps_in_epoch:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()

            progress_bar.set_postfix({"loss": f"{loss.item():.4f}"})

    return total_loss / len(data_loader)


## Evaluate

In [ ]:
def evaluate(model, data_loader, loss_fn, device):
    """Đánh giá mô hình trên tập dữ liệu."""
    model.eval()
    all_preds = []
    all_labels = []
    total_val_loss = 0

    progress_bar = tqdm(data_loader, desc="Evaluating", leave=False, dynamic_ncols=True)

    with torch.no_grad(), logging_redirect_tqdm():
        for batch in progress_bar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # <<< TÍNH LOSS TRÊN TẬP VALIDATION
            loss = loss_fn(logits, labels)
            total_val_loss += loss.item()

            preds = torch.argmax(outputs.logits, dim=-1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(data_loader)  # <<< TÍNH LOSS TRUNG BÌNH
    return all_labels, all_preds, avg_val_loss  # <<< TRẢ VỀ THÊM LOSS


# Main

In [ ]:
# Tải biến môi trường từ file envs/.env.
dotenv_path = os.path.join(os.getcwd(), "envs", ".env")
load_dotenv(dotenv_path)
print(f"dotenv_path: {dotenv_path}")


dotenv_path: /home/guest/Projects/CS221/envs/.env


In [ ]:
# lấy HF token để login
hf_token = os.getenv("HUGGING_FACE_TOKEN")

if hf_token:
    print("INFO: Tìm thấy HUGGING_FACE_TOKEN. Đang đăng nhập...")
    login(token=hf_token)
    print("INFO: Đăng nhập Hugging Face thành công.")
else:
    print(
        "WARNING: Không tìm thấy HUGGING_FACE_TOKEN trong file .env. Một số model có thể yêu cầu đăng nhập."
    )


INFO: Tìm thấy HUGGING_FACE_TOKEN. Đang đăng nhập...
INFO: Đăng nhập Hugging Face thành công.


## 1. Chuẩn bị dữ liệu

In [ ]:
logger.info("Bắt đầu pipeline huấn luyện.")

# 1. Chuẩn bị dữ liệu
logger.info("Bước 1: Chuẩn bị dữ liệu...")
train_df, val_df = prepare_data(cfg, logger=logger)
if train_df is None:
    logger.error("Dữ liệu không thể được chuẩn bị. Dừng chương trình.")


2025-10-17 13:42:27 - [INFO] - Bắt đầu pipeline huấn luyện.
2025-10-17 13:42:27 - [INFO] - Bước 1: Chuẩn bị dữ liệu...
2025-10-17 13:42:27 - [INFO] - Chia dữ liệu: 5600 mẫu train, 1400 mẫu validation.


✅ Đọc thành công 7000 mẫu từ file đã xử lý: /home/guest/Projects/CS221/data/vihallu-train.csv
✅ Đã lưu tập train vào: /home/guest/Projects/CS221/data/processed/train_split.csv
✅ Đã lưu tập validation vào: /home/guest/Projects/CS221/data/processed/validation_split.csv


## 2. Tải model và tokenizer

In [ ]:
# --- SỬA LẠI LỜI GỌI HÀM TRONG NOTEBOOK ---
# Cell [13] cần sửa lại để truyền logger vào
logger.info(f"Bước 2: Tải model '{cfg.MODEL_NAME}' và tokenizer...")
model, tokenizer = get_model_and_tokenizer(cfg, logger)  # Truyền logger vào


2025-10-17 13:42:27 - [INFO] - Bước 2: Tải model 'FacebookAI/xlm-roberta-large' và tokenizer...


Đang tải model: FacebookAI/xlm-roberta-large
Model config: XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}



Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# %pip install torchinfo


In [ ]:
from torchinfo import summary
import torch  # Đảm bảo đã import torch

logger.info("Phân tích kiến trúc mô hình bằng torchinfo...")

# --- Dùng torchinfo để hiển thị ---
# Tạo một input giả với batch_size và max_length như trong config
input_ids_example = torch.randint(
    0, tokenizer.vocab_size, (cfg.BATCH_SIZE, cfg.MAX_LENGTH)
)

# 1. Gọi summary với verbose=0 để không in ra console và lưu kết quả vào biến
#    Thêm các cột bạn muốn xem, ví dụ: 'output_size', 'num_params'
model_summary = summary(
    model,
    input_data={"input_ids": input_ids_example},
    verbose=0,  # <-- Quan trọng: Ngăn không cho tự động in
    col_names=["input_size", "output_size", "num_params", "mult_adds"],
)

# 2. Chuyển đối tượng summary thành string và đưa vào logger
logger.info(f"Kiến trúc chi tiết của mô hình:\n{str(model_summary)}")


# # (Tùy chọn) Bạn vẫn có thể in ra màn hình nếu muốn xem ngay trong notebook
# print("In summary ra màn hình notebook:")
# print(model_summary)


2025-10-17 13:42:29 - [INFO] - Phân tích kiến trúc mô hình bằng torchinfo...
2025-10-17 13:42:32 - [INFO] - Kiến trúc chi tiết của mô hình:
Layer (type:depth-idx)                                            Input Shape               Output Shape              Param #                   Mult-Adds
XLMRobertaForSequenceClassification                               --                        [4, 3]                    --                        --
├─XLMRobertaModel: 1-1                                            [4, 512]                  [4, 512, 1024]            --                        --
│    └─XLMRobertaEmbeddings: 2-1                                  --                        [4, 512, 1024]            --                        --
│    │    └─Embedding: 3-1                                        [4, 512]                  [4, 512, 1024]            256,002,048               1,024,008,192
│    │    └─Embedding: 3-2                                        [4, 512]                  [4, 512, 1024] 

## 3. Tạo Dataset và DataLoader

In [ ]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding  # <-- 1. Import DataCollator

# --- TẠO DATASET VÀ DATALOADER ---
logger.info("Bước 3: Tạo Dataset và DataLoader...")

# Tạo Dataset (với class HallucinationDataset đã được chỉnh sửa ở trên)
train_dataset = HallucinationDataset(
    premises=train_df["premise"].to_list(),
    hypotheses=train_df["hypothesis"].to_list(),
    labels=train_df["label_id"].to_list(),
    tokenizer=tokenizer,
    max_len=cfg.MAX_LENGTH,
)
val_dataset = HallucinationDataset(
    premises=val_df["premise"].to_list(),
    hypotheses=val_df["hypothesis"].to_list(),
    labels=val_df["label_id"].to_list(),
    tokenizer=tokenizer,
    max_len=cfg.MAX_LENGTH,
)

# 3. Tạo một instance của DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 4. Tạo DataLoader và truyền data_collator vào
train_loader = DataLoader(
    train_dataset,
    batch_size=cfg.BATCH_SIZE,
    shuffle=True,
    collate_fn=data_collator,  # <-- Dùng data_collator ở đây
)
val_loader = DataLoader(
    val_dataset,
    batch_size=cfg.BATCH_SIZE,
    collate_fn=data_collator,  # <-- Dùng data_collator ở đây
)

logger.info("✅ Tạo DataLoader thành công với DataCollatorWithPadding chuẩn!")


2025-10-17 13:42:32 - [INFO] - Bước 3: Tạo Dataset và DataLoader...
2025-10-17 13:42:32 - [INFO] - ✅ Tạo DataLoader thành công với DataCollatorWithPadding chuẩn!
2025-10-17 13:42:32 - [INFO] - ✅ Tạo DataLoader thành công với DataCollatorWithPadding chuẩn!


In [ ]:
gradient_accumulation_steps = max(1, cfg.GRADIENT_ACCUMULATION_STEPS)
effective_batch_size = cfg.BATCH_SIZE * gradient_accumulation_steps
logger.info(
    "Gradient accumulation steps: %s | Effective batch size: %s",
    gradient_accumulation_steps,
    effective_batch_size,
)


2025-10-17 13:42:32 - [INFO] - Gradient accumulation steps: 4 | Effective batch size: 16


### Check dataset

In [ ]:
print("\n--- Kiểm tra 1 batch dữ liệu đầu vào ---")
sample_batch = next(iter(train_loader))

print("Kích thước input_ids:", sample_batch["input_ids"].shape)
print("Kích thước attention_mask:", sample_batch["attention_mask"].shape)
print("Nhãn trong batch:", sample_batch["labels"])

# Giải mã một mẫu để xem nó trông như thế nào
decoded_text = tokenizer.decode(sample_batch["input_ids"][0], skip_special_tokens=False)
print("\nMột mẫu đã được token hóa và giải mã lại:")
print(decoded_text)
print("------------------------------------------\n")
# --- KẾT THÚC BƯỚC KIỂM TRA ---



--- Kiểm tra 1 batch dữ liệu đầu vào ---
Kích thước input_ids: torch.Size([4, 367])
Kích thước attention_mask: torch.Size([4, 367])
Nhãn trong batch: tensor([2, 2, 0, 0])

Một mẫu đã được token hóa và giải mã lại:
<s> Câu hỏi: Bức xạ nhiệt từ mặt đất ngược trở lại không khí ảnh hưởng như thế nào đến với tầng đối lưu? Ngữ cảnh: Nguyên nhân các biến đổi nhiệt độ trong tầng đối lưu là do nhiệt độ được xác định bởi bức xạ nhiệt từ mặt đất ngược trở lại không khí. Mặc dù tia nắng Mặt Trời tiếp xúc với phần không khí ở trên cao trước, nhưng không khí khá trong suốt nghĩa là nó hấp thụ rất ít năng lượng của tia nắng. Đa phần năng lượng Mặt Trời rơi xuống mặt đất, tại đây, nó bị hấp thụ mạnh bởi mặt đất, và làm mặt đất nóng lên (nóng hơn không khí trên cao). Mặt đất nóng truyền nhiệt trực tiếp cho lớp không khí gần mặt đất; không khí gần mặt đất nóng lên và nở ra, nhẹ hơn phần không khí lạnh ở trên và bay lên cao nhờ lực đẩy Ácsimét. Khi không khí nóng bay lên cao, nó giãn nở đoạn nhiệt nghĩa

In [ ]:
print("\n--- Kiểm tra chi tiết 5 mẫu đầu tiên để so sánh trước và sau khi xử lý ---")

# Lấy 5 mẫu đầu tiên từ DataFrame gốc để so sánh
num_samples_to_check = 5
for i in range(num_samples_to_check):
    print(f"\n=============== MẪU {i} ===============")

    # 1. Lấy dữ liệu gốc từ DataFrame
    original_premise = train_df["premise"].iloc[i]
    original_hypothesis = train_df["hypothesis"].iloc[i]
    # Nối 2 chuỗi lại giống cách tokenizer sẽ thấy chúng
    original_combined_text = original_premise + " [SEP] " + original_hypothesis

    # 2. Lấy dữ liệu đã được xử lý từ Dataset
    processed_sample = train_dataset[i]
    processed_input_ids = processed_sample["input_ids"]

    # 3. Giải mã (decode) các input_ids đã xử lý trở lại thành văn bản
    decoded_text = tokenizer.decode(processed_input_ids, skip_special_tokens=False)

    # 4. So sánh và in kết quả
    original_token_count = len(tokenizer.encode(original_premise, original_hypothesis))
    processed_token_count = len(processed_input_ids)

    print(f"Số token gốc (ước tính): {original_token_count}")
    print(
        f"Số token sau khi xử lý (giới hạn bởi max_len={cfg.MAX_LENGTH}): {processed_token_count}"
    )

    if original_token_count > cfg.MAX_LENGTH:
        print("⚠️  CẢNH BÁO: Mẫu này đã bị cắt bớt (truncated)!")
    else:
        print("✅  OK: Độ dài mẫu nằm trong giới hạn, không bị cắt.")

    print("\n--- Văn bản GỐC  ---")
    print(original_combined_text)

    print("\n--- Văn bản SAU KHI DECODE từ input_ids ---")
    print(decoded_text)

print("\n===========================================")
print(
    "Kiểm tra hoàn tất. Hãy so sánh văn bản trên để xem có sự khác biệt ở cuối chuỗi không."
)



--- Kiểm tra chi tiết 5 mẫu đầu tiên để so sánh trước và sau khi xử lý ---

=============== MẪU 0 ===============
Số token gốc (ước tính): 280
Số token sau khi xử lý (giới hạn bởi max_len=512): 280
✅  OK: Độ dài mẫu nằm trong giới hạn, không bị cắt.

--- Văn bản GỐC  ---
Câu hỏi: Chiến sự năm 1950 đã dánh bại quân Hoa Kỳ o chiên trạn nào? Ngữ cảnh: Từ ngày 25 tháng 10 đến ngày 5 tháng 11 (1950) là chiến dịch đầu tiên của Trung Quốc. Quân Trung Quốc dùng 2 sư đoàn của quân đoàn 42 tổ chức phòng ngự ở khu vực Hoàng Thảo Lĩnh, Phó Chiến Lĩnh thuộc mặt trận miền đông, lại dùng 3 quân đoàn và một sư đoàn của quân đoàn 42 (sau tăng thêm 2 quân đoàn) phản kích ở mặt trận miền Tây. Chiến dịch này đã đánh lui quân Mỹ đến phía nam sông Thanh Xuyên. Ngày 7 tháng 11, các quân đoàn 20, 26, 27 thuộc Binh đoàn 9 quân Chí nguyện dưới quyền chỉ huy của Tư lệnh kiêm Chính uỷ Tống Thời Luân tiến vào Triều Tiên. Tới lúc này, binh lực tác chiến của Trung Quốc lên tới 9 quân đoàn gồm 30 sư đoàn, tổng cộng 

## 4. Thiết lập Huấn luyện

In [ ]:
import torch  # Đảm bảo đã import torch

logger.info("Bước 4: Thiết lập môi trường huấn luyện và kiến trúc model...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Log thông tin thiết bị (GPU/CPU) ---
logger.info(f"Sử dụng thiết bị: {device}")
if device.type == "cuda":
    gpu_count = torch.cuda.device_count()
    gpu_name = torch.cuda.get_device_name(0)
    logger.info(f"✅ Tìm thấy {gpu_count} GPU(s).")
    logger.info(f"✅ Đang sử dụng GPU: {gpu_name}")
else:
    logger.warning("⚠️ Không tìm thấy GPU, sử dụng CPU. Quá trình training sẽ rất chậm.")

# --- BẮT ĐẦU PHẦN THÊM MỚI ---
# Chuyển toàn bộ kiến trúc model thành dạng string để đưa vào logger
model_architecture_string = str(model)

# Ghi log kiến trúc model
logger.info(f"Kiến trúc của mô hình:\n{model_architecture_string}")
# --- KẾT THÚC PHẦN THÊM MỚI ---

# Di chuyển model đến device đã chọn
model.to(device)


2025-10-17 13:42:32 - [INFO] - Bước 4: Thiết lập môi trường huấn luyện và kiến trúc model...
2025-10-17 13:42:32 - [INFO] - Sử dụng thiết bị: cuda
2025-10-17 13:42:32 - [INFO] - ✅ Tìm thấy 1 GPU(s).
2025-10-17 13:42:32 - [INFO] - ✅ Đang sử dụng GPU: NVIDIA GeForce RTX 5070 Ti
2025-10-17 13:42:32 - [INFO] - Kiến trúc của mô hình:
XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=102

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, ou

In [ ]:
optimizer = AdamW(
    model.parameters(),
    lr=cfg.LEARNING_RATE,
    weight_decay=cfg.WEIGHT_DECAY,
    eps=cfg.EPSILON,
)


In [ ]:
from transformers import (
    get_scheduler,
    get_cosine_with_hard_restarts_schedule_with_warmup,
)

num_update_steps_per_epoch = math.ceil(len(train_loader) / gradient_accumulation_steps)
num_training_steps = num_update_steps_per_epoch * cfg.EPOCHS
logger.info(
    "Scheduler will run for %s total steps (%s per epoch)",
    num_training_steps,
    num_update_steps_per_epoch,
)

if cfg.TOTAL_STEP_SCALE <= 0:
    warmup_steps = 0
elif cfg.TOTAL_STEP_SCALE <= 1:
    warmup_steps = max(1, int(cfg.TOTAL_STEP_SCALE * num_training_steps))
else:
    warmup_steps = min(int(cfg.TOTAL_STEP_SCALE), num_training_steps)

# <<< THÊM KHỐI LỆNH IF ĐỂ XỬ LÝ TRƯỜNG HỢP ĐẶC BIỆT
if cfg.SCHEDULER_TYPE == "cosine_with_restarts":
    logger.info(
        f"Sử dụng scheduler chuyên dụng: cosine_with_hard_restarts_schedule_with_warmup với {cfg.NUM_CYCLES} chu kỳ."
    )
    scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=num_training_steps,
        num_cycles=cfg.NUM_CYCLES,  # <-- Tham số chuyên biệt hoạt động ở đây!
    )
else:
    # Giữ lại hàm get_scheduler chung cho tất cả các loại scheduler khác
    logger.info(f"Sử dụng scheduler chung: {cfg.SCHEDULER_TYPE}")
    scheduler = get_scheduler(
        cfg.SCHEDULER_TYPE,
        optimizer=optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=num_training_steps,
    )

logger.info("Warmup steps: %s", warmup_steps)


2025-10-17 13:42:32 - [INFO] - Scheduler will run for 3500 total steps (350 per epoch)
2025-10-17 13:42:32 - [INFO] - Sử dụng scheduler chung: cosine
2025-10-17 13:42:32 - [INFO] - Warmup steps: 350
2025-10-17 13:42:32 - [INFO] - Sử dụng scheduler chung: cosine
2025-10-17 13:42:32 - [INFO] - Warmup steps: 350


In [ ]:
# Chuyển class weights từ config thành tensor và đưa lên device
if cfg.CLASS_WEIGHTS:
    logger.info("Sử dụng Class Weights & Label smoothing cho hàm loss.")
    class_weights_tensor = torch.tensor(cfg.CLASS_WEIGHTS, dtype=torch.float).to(device)
    loss_fn = torch.nn.CrossEntropyLoss(
        weight=class_weights_tensor,
        label_smoothing=cfg.LABEL_SMOOTHING,
    ).to(device)
else:
    logger.info("Sử dụng CrossEntropyLoss thông thường (không có trọng số).")
    loss_fn = torch.nn.CrossEntropyLoss().to(device)


2025-10-17 13:42:32 - [INFO] - Sử dụng Class Weights & Label smoothing cho hàm loss.


## 5. Vòng lặp Huấn luyện

In [ ]:
best_macro_f1 = 0.0
patience_counter = 0  # bien dem => early stopped khi f1 ko tang them => overfitting

for epoch in range(cfg.EPOCHS):
    logger.info(f"--- Epoch {epoch + 1}/{cfg.EPOCHS} ---")

    avg_train_loss = train_one_epoch(
        model,
        train_loader,
        loss_fn,
        optimizer,
        scheduler,
        device,
        epoch + 1,
        cfg.EPOCHS,
        gradient_accumulation_steps=gradient_accumulation_steps,
    )
    logger.info(f"Loss trung bình trên tập train: {avg_train_loss:.4f}")

    current_lr = optimizer.param_groups[0]["lr"]
    logger.info(
        f"Current Learning Rate: {current_lr:.2e}"
    )  # Dùng định dạng khoa học e.g., 8.00e-06

    # Đánh giá trên tập validation
    logger.info("Bắt đầu đánh giá trên tập validation...")
    val_labels, val_preds, avg_val_loss = evaluate(model, val_loader, loss_fn, device)

    accuracy = accuracy_score(val_labels, val_preds)
    macro_f1 = f1_score(val_labels, val_preds, average="macro")

    logger.info(f"Validation Loss: {avg_val_loss:.4f}")
    logger.info(f"Validation Accuracy: {accuracy:.4f}")
    logger.info(f"Validation Macro-F1: {macro_f1:.4f}")

    # # In classification report chi tiết
    target_names = [cfg.ID2LABEL[i] for i in range(len(cfg.LABEL_MAP))]

    # In classification report chi tiết (có thể giữ lại print hoặc log từng dòng)
    report = classification_report(
        val_labels,
        val_preds,
        target_names=[cfg.ID2LABEL[i] for i in range(len(cfg.LABEL_MAP))],
        digits=4,
    )
    logger.info(f"Classification Report trên tập validation:\n{report}")

    # Lưu lại model tốt nhất dựa trên Macro-F1
    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        patience_counter = 0  # << RESET BỘ ĐẾM

        logger.info(
            f"🎉 Macro-F1 cải thiện. Đang lưu model tốt nhất vào '{cfg.MODEL_OUTPUT_DIR}'..."
        )
        if not os.path.exists(cfg.MODEL_OUTPUT_DIR):
            os.makedirs(cfg.MODEL_OUTPUT_DIR)

        model.save_pretrained(cfg.MODEL_OUTPUT_DIR)
        tokenizer.save_pretrained(cfg.MODEL_OUTPUT_DIR)
        logger.info("Lưu model thành công.")
    else:
        patience_counter += 1
        logger.warning(
            f"Macro-F1 không cải thiện. Patience: {patience_counter}/{cfg.PATIENCE_LIMIT}"
        )
        if patience_counter >= cfg.PATIENCE_LIMIT:
            logger.info("Early stopping! Dừng huấn luyện.")
            break


2025-10-17 13:42:32 - [INFO] - --- Epoch 1/10 ---


Epoch 1/10:   0%|          | 0/1400 [00:00<?, ?it/s]

2025-10-17 13:46:59 - [INFO] - Loss trung bình trên tập train: 1.0761
2025-10-17 13:46:59 - [INFO] - Current Learning Rate: 8.00e-06
2025-10-17 13:46:59 - [INFO] - Bắt đầu đánh giá trên tập validation...


Evaluating:   0%|          | 0/350 [00:00<?, ?it/s]

2025-10-17 13:47:19 - [INFO] - Validation Loss: 0.9351
2025-10-17 13:47:19 - [INFO] - Validation Accuracy: 0.5850
2025-10-17 13:47:19 - [INFO] - Validation Macro-F1: 0.5267
2025-10-17 13:47:19 - [INFO] - Classification Report trên tập validation:
              precision    recall  f1-score   support

          no     0.6561    0.7862    0.7153       449
   extrinsic     0.5313    0.8829    0.6634       461
   intrinsic     0.6146    0.1204    0.2014       490

    accuracy                         0.5850      1400
   macro avg     0.6007    0.5965    0.5267      1400
weighted avg     0.6005    0.5850    0.5183      1400

2025-10-17 13:47:19 - [INFO] - 🎉 Macro-F1 cải thiện. Đang lưu model tốt nhất vào '/home/guest/Projects/CS221/models/xlm-roberta-large-tuned'...
2025-10-17 13:47:22 - [INFO] - Lưu model thành công.
2025-10-17 13:47:22 - [INFO] - --- Epoch 2/10 ---


...

Epoch 3/10:   0%|          | 0/1400 [00:00<?, ?it/s]

2025-10-17 13:56:36 - [INFO] - Loss trung bình trên tập train: 0.6770
2025-10-17 13:56:36 - [INFO] - Current Learning Rate: 7.06e-06
2025-10-17 13:56:36 - [INFO] - Bắt đầu đánh giá trên tập validation...


Evaluating:   0%|          | 0/350 [00:00<?, ?it/s]

2025-10-17 13:56:57 - [INFO] - Validation Loss: 0.6567
2025-10-17 13:56:57 - [INFO] - Validation Accuracy: 0.7800
2025-10-17 13:56:57 - [INFO] - Validation Macro-F1: 0.7799
2025-10-17 13:56:57 - [INFO] - Classification Report trên tập validation:
              precision    recall  f1-score   support

          no     0.7800    0.8686    0.8219       449
   extrinsic     0.7981    0.7462    0.7713       461
   intrinsic     0.7633    0.7306    0.7466       490

    accuracy                         0.7800      1400
   macro avg     0.7805    0.7818    0.7799      1400
weighted avg     0.7801    0.7800    0.7789      1400

2025-10-17 13:56:57 - [INFO] - 🎉 Macro-F1 cải thiện. Đang lưu model tốt nhất vào '/home/guest/Projects/CS221/models/xlm-roberta-large-tuned'...
2025-10-17 13:56:59 - [INFO] - Lưu model thành công.
2025-10-17 13:56:59 - [INFO] - --- Epoch 4/10 ---


Epoch 4/10:   0%|          | 0/1400 [00:00<?, ?it/s]

In [ ]:
logger.info("🏁 Quá trình huấn luyện hoàn tất.")
logger.info(
    f"Model tốt nhất với Macro-F1 = {best_macro_f1:.4f} đã được lưu tại '{cfg.MODEL_OUTPUT_DIR}'"
)


# Phân phối kết quả đúng/sai theo từng lớp

In [ ]:
val_label_names = [cfg.ID2LABEL[label_id] for label_id in val_labels]
pred_label_names = [cfg.ID2LABEL[pred_id] for pred_id in val_preds]
evaluation_df = pd.DataFrame(
    {
        "true_label": val_label_names,
        "predicted_label": pred_label_names,
    }
)
evaluation_df["status"] = evaluation_df.apply(
    lambda row: (
        "correct" if row["true_label"] == row["predicted_label"] else "incorrect"
    ),
    axis=1,
)
distribution_table = (
    evaluation_df.groupby(["true_label", "status"])
    .size()
    .unstack(fill_value=0)
    .rename_axis(None, axis=1)
    .reset_index()
    .sort_values("true_label")
)

# 1. Thêm cột 'total' bằng cách cộng cột 'correct' và 'incorrect'
distribution_table["total"] = (
    distribution_table["correct"] + distribution_table["incorrect"]
)

# 2. Thêm cột tỉ lệ đúng (correct_rate)
distribution_table["correct_rate"] = (
    distribution_table["correct"] / distribution_table["total"]
)

# 3. Thêm cột tỉ lệ sai (incorrect_rate)
distribution_table["incorrect_rate"] = (
    distribution_table["incorrect"] / distribution_table["total"]
)

# (Tùy chọn) Format các cột tỉ lệ thành dạng phần trăm cho dễ đọc
distribution_table["correct_rate"] = distribution_table["correct_rate"].map(
    "{:.2%}".format
)
distribution_table["incorrect_rate"] = distribution_table["incorrect_rate"].map(
    "{:.2%}".format
)

# In ra bảng kết quả
logger.info(f"Phân phối kết quả trên từng lớp:\n{distribution_table.to_string()}")

# Trong notebook, dùng display() sẽ cho bảng đẹp hơn
print("Bảng phân phối kết quả trên từng lớp:")
display(distribution_table)
